# Approssimazione di $e^{sin(t-1)}$

Il notebook è diviso in due sezioni, la prima che interpola $e^{sin(t-1)}$ con un modello polinomiale che si sviluppa fino alla potenza settima; la seconda sezione interpola ancora $e^{sin(t-1)}$ ma con un modello di sviluppo in serie di Fourier del 3° ordine (5 coefficienti, uno per il termine costante, due per i due termini $cos(x)$ e $sin(x)$, due per i due termini $cos(2x)$ e $sin(2x)$) \
Per la risoluzione di questo esercizio sono riportati tre modi, che utilizzano sempre il metodo dei minimi quadrati.
- 1° metodo (commentato): si genera la matrice di regressione in maniera esplicita (colonna per colonna) e poi si applica il metodo dei minimi quadrati
- 2° metodo (effettivamente utilizzato): la matrice di regressione è generata nella maniera più compatta possibile, utilizzando hcat
- 3° metodo (commentato): si utilizzano funzioni definite nel file interpolation.jl. Hanno il vantaggio di poter cambiare il numero di coefficienti dello sviluppo, sia polinomiale che di Fourier. 

In [ ]:
#Import cell
using Markdown     #Visualizzazione di markdown in display
using Plots        #Per grafici
using Printf       #Per le stringhe del grafico
using LaTeXStrings
#non devo importare LinearAlgebra perchè è già dentro il file linear_systems.jl, che è dentro a interpolation.jl
include("c:\\ALL\\Stefano\\Bicocca\\3terzo_anno\\lab_comp\\lab_computazionale1\\librerie\\interpolation.jl")

In [ ]:
function exact_func(t)
    return exp(sin(t-1))
end

function model1(t, c0, c1, c2, c3, c4, c5, c6)
    return c0 + c1*t + c2*t^2 + c3*t^3 + c4*t^4 + c5*t^5 + c6*t^6
end

function model2(t, d1, d2, d3, d4, d5)
    return d1 + d2*cos(t) + d3*sin(t) + d4*cos(2t) +d5*sin(2t)
end

## Interpolazione 1
Modello:  $y = c_0 + c_1t + c_2t^2 + c_3t^3 + c_4t^4 + c_5t^5 + c_6t^6$

In [ ]:
#Generazione dei dati acquisiti
t = [2*pi*i/60.0 for i in 1:60]
y_meas = exact_func.(t)

m = minimum(t)
M = maximum(t)
x = [i for i in m:0.1:M]      #Vettore per generare funzioni continue

# Generazione di A in maniera estesa
#= A = zeros(length(t), 7)     #righe = n° di misure, colonne = n° di coefficienti.
A[:,1] = ones(length(t), 1)
A[:,2] = t
A[:,3] = t.^2
A[:,4] = t.^3
A[:,5] = t.^4
A[:,6] = t.^5
A[:,7] = t.^6 =#

#= Generazione di A in maniera compatta
[t.^(i-1) for i in 1:7] è un array di 7 colonne, ognuna delle quali è un vettore colonna
A = hcat([t.^(i-1) for i in 1:7]...) #hcat concatena le colonne
... è un operatore che spacchetta l'array in modo che hcat possa concatenare le colonne =#
A = hcat([t.^(i-1) for i in 1:7]...) 

c = least_sq(A, y_meas)
y_interp = model1.(x, c[1], c[2], c[3], c[4], c[5], c[6], c[7])

#= #Soluzione con l'utilizzo della libreria interpolation.jl
n_coefficienti = 7
c = pol_fit(t, y_meas, n_coefficienti)
y_interp = pol_expansion(x, c) =#


In [ ]:
fig1 = plot(title = L"Interpolation of $e^{sin(t-1)}$",
            xlabel = "Time",
            ylabel = "g(t)")

scatter!(fig1,
      t,
      y_meas,
      label="Dati raccolti"
      )

plot!(fig1,
      x,
      y_interp,
      label="Interpolazione")
            
display(fig1)

## Interpolazione 2
Modello:  $y = d_1 + d_2cos(t) + d_3sin(t) + d_4cos(2t) + d_5sin(2t)$

In [ ]:
#Generazione di A in maniera estesa
#= A = zeros(length(t), 5)     #righe = n° di misure, colonne = n° di coefficienti.
A[:,1] = ones(length(t), 1)
A[:,2] = cos.(t)
A[:,3] = sin.(t)
A[:,4] = cos.(2t)
A[:,5] = sin.(2t) =#

#Generazione di A in forma compatta
A = hcat(ones(60, 1), cos.(t), sin.(t), cos.(2t), sin.(2t))

c = least_sq(A, y_meas)
y_interp = model2.(x, c[1], c[2], c[3], c[4], c[5])

#= #Soluzione con l'utilizzo della libreria interpolation.jl
c = fourier_fit(t, y_meas, 5)
y_interp = fourier_expansion(x, c) =#


In [ ]:
fig1 = plot(title = L"Interpolation of $e^{sin(t-1)}$",
            xlabel = "Time",
            ylabel = "g(t)")

scatter!(fig1,
      t,
      y_meas,
      label="Dati raccolti"
      )

plot!(fig1,
      x,
      y_interp,
      label="Interpolazione")
            
display(fig1)